<a href="https://colab.research.google.com/github/xspronken/bigdataW1/blob/master/week3v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math as m


''' For conversion to natural units'''
G = 6.67*10**(-11) # (m**3)*(kg**-1)*(s**-2)
c = 3*10**8 # m/s

'''variables'''
SNR = 10
m1 = 10**30  * G/c**2# kg
m2 = 10**30 * G/c**2 # kg

M = (m1+m2) # kg
n = m1*m2/(M**2) #dimensionless
M_chirp = n**(3/5)*M # kg
t_c = 100/c# secs
phi_c = 100 #rads
print (M)
params = [t_c, phi_c, M_chirp, n]

alpha = [1, 0, (20/9*(743/336 + n*11/4)), -16*m.pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
PN = 1



''' defining variables for Power Spectral Density S(f) of Initial Ligo '''
fs = 40 # Hz
f0 = 150 # Hz
s0 = 9**(-46) #Hz**-1

''' defining Power Spectral Density S(f) of Initial Ligo '''
def S(f):
  x = f/f0
  if (f >= fs):
    return s0*((4.49*x)**(-56)+0.16*x**(-4.52)+0.52+0.32*x**2)
  else: 
    return float('inf')

''' defining upper and lower limit fo the integrations''' 
F_high = (6**(3/2)*m.pi*M*G/(c**3))**-1 # assuming the to constants to be dimensionless and M being in kg we multiply by g/c**3 to get Hz as a final result
F_low = fs
print ('f high ' , F_high)

''' getting the variable A from integration '''
def A2(SNR, dx):
  #perform integral using trapezoidal sums
  accumulator = 0
  for i in range (int(F_low/dx), int(F_high/dx - 1)):
   x = i*dx
   fi = ((x)**(-7/6) / (S(x)))
   fi2 = ((x + dx)**(-7/6) / (S(x + dx)))
   accumulator += dx * (fi + fi2)/2
  # divide SNR by integral multiplied by 4
  return SNR/(4*accumulator)

A = m.sqrt(A2(SNR, 0.001))
print(A)


''' defining the Fourrier transform h(f)'''
def h(f, params, x):
  #defining the waveform's variables
  t_c, phi_c, M_chirp , n = params
  
  v = (m.pi*M*f*G/c**3)**(1/3) # v is dimensionless, M in kg, F in Hz, multiplying by G/c**3 converts to the right units
  
  #sum of all coefficients (alpha) in the fourrier phase
  x *= 2
  sum_alpha = 0
  for i in range(x):
    sum_alpha += (alpha[i] * (v**i))
  
  #defining the waveform psi(f)
  psi =  2 * m.pi * f * t_c - phi_c - m.pi/4 + 3/(128*n*v**5) * sum_alpha

  return  A * f**(-7/6) * m.e**(1j * psi)


''' Calculating the partial derivative of h(f) '''
def h_partial(f, params, x, dx):   
  dparams = params.copy()
  dparams[x] += dx
  dy = h(f, dparams, PN)-h(f, params, PN)
  return dy/dx

''' Generating the Fischer information matrix '''
def generate_matrix(params, dx):
  
# create a  n*n matrix where n is the number of observables
  n = len(params)
  matrix = np.empty([n,n]) 
  
# Start filling the matrix
  for i in range (n):
    for j in range (n):
      
      #start integration (again with trap sums)
      accumulator = 0
      for k in range(int(F_low/dx), int(F_high/dx-1)):
        x1 = k*dx
        x2 = k*dx + dx 
        part_i1 = h_partial(x1, params,i, 0.001)
        part_j1 = np.conj(h_partial(x1, params,j, 0.001))
        f1 = part_i1 * part_j1/S(x1)
        
        part_i2 = h_partial(x2, params,i, 0.001)
        part_j2 = np.conj(h_partial(x2, params,j, 0.001))
        f2 = part_i2 * part_j2/S(x2) 
        
        accumulator += np.real(dx * (f1 + f2)/2)
      
      #once integration is done add it to the matrix
      matrix[i][j] = accumulator*4
  print(matrix)
  return matrix


''' Extract the diagonal elements from the inverted Fischer matrix'''

def extract_from_matrix(matrix):
  #invert matrix
  y = np.linalg.inv(matrix) 
  z = []
  
  #fetch the square root of diagonal elements
  for i in range (len(y)):
    z.append(m.sqrt(y[i][i]))
  
  return z

test = extract_from_matrix(generate_matrix(params, 1))

In [17]:
print(test)
print (M_chirp)
print(n)

[0.0060462174814064585, 6.64784788235983, 0.0039041263510136127, 1146644716.5566647]
2.1505823174759808e-06
0.25
